In [1]:
import evaluate
from openai import AzureOpenAI
import os
import pandas as pd
import datasets
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import ParameterGrid
import evaluate

client = AzureOpenAI(azure_endpoint='https://oai-cbipm-01.openai.azure.com/',
                     api_key=os.environ["OPENAI_API_KEY"],
                     api_version="2023-12-01-preview")  # 2023-12-01-preview <- highest version number as of 1/12/23

deployment = "Deployment"

env: OPENAI_API_KEY=cb0e51bd1e4a46fbb4043b8bcdd04cd7


In [2]:
# metrics function
def compute_metrics(predictions, references, labels=None, pos_label=1, average="weighted", sample_weight=None, zero_division='warn'):
        f1 = f1_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight
        )
        p = precision_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight,
            zero_division=zero_division
        )
        r = recall_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight,
            zero_division=zero_division
        )
        c = classification_report(
            references, predictions, labels=labels
        )
        print(c)
        return {"f1": float(f1) if f1.size == 1 else f1,
                "precision": float(p) if p.size == 1 else p,
                "recall": float(r) if r.size == 1 else r}

In [9]:
train = pd.read_csv("../data/train_PT_MD_labels.csv")
test = pd.read_csv("../data/test_PT_MD_labels.csv")

json_train_sentences = train["language"].to_json()
json_train_labels = train["PT_MD_labels"].to_json()

# converting test sentences to json format
json_test_sentences = test["language"].to_json()

messages = [{"role": "system", "content": "You are a medical doctor."},
            {"role": "user", "content": "As a medical doctor, you write many clinical notes about patients.\n"
                                        "When writing these clinical notes, you try to be as neutral as possible, but sometimes what you right might offend a patient even if you don't intend it to.\n"
                                        "Your task is to analyze the sentiment of a series of sentences you wrote about patients.\n"
                                        "For each sentence, answer two questions:\n"
                                        "First, as the doctor writing the note, what is your attitude towards the patient you wrote about?\n"
                                        "Secondly, if you were the patient being written about, how would you feel reading this description of you?\n"
                                        "Please assign a sentiment score of negative, neutral, or positive for each sentence.\n"
                                        "Below are three example sentences in JSON format:\n"
                                        f"{json_train_sentences}"
                                        "Please provide your answer in JSON format."},
            {"role": "assistant", "content": f"{json_train_labels}"},
            {"role": "user", "content": f"Complete the same task with each of these sentences:\n{json_test_sentences}"}]

response = client.chat.completions.create(model=deployment, messages=messages, temperature=0, seed=42)

output = response.choices[0].message.content
print(output)

predictions = pd.read_json(output, orient="index")
results = compute_metrics(predictions[0], test["PT_MD_labels"])
print(results)

i = 0
errors = {'Errors': []}
for pred_lab, true_lab in zip(predictions[0], test["PT_MD_labels"]):
    if pred_lab != true_lab:
        errors['Errors'].append((test["language"][i],
                            pred_lab,
                            true_lab))
        i += 1

with open(f'error_analysis.tsv', 'w') as f:
    for k, vect in errors.items():
        if k == 'Errors':
            for sen in vect:
                f.write(sen[0] + '\t' + f'PRED_{sen[1].upper()}' + '\t' + f'TRUE_{sen[2].upper()}' + '\n')
            f.write('\n')

{"0":"negative_negative","1":"negative_negative","2":"negative_negative","3":"negative_negative","4":"negative_negative","5":"negative_negative","6":"negative_negative","7":"negative_negative","8":"negative_negative","9":"negative_negative","10":"neutral_negative","11":"negative_negative","12":"positive_positive","13":"neutral_neutral","14":"neutral_neutral","15":"positive_positive","16":"negative_negative","17":"positive_neutral","18":"negative_negative","19":"neutral_neutral","20":"negative_negative","21":"neutral_neutral","22":"neutral_neutral","23":"negative_negative","24":"neutral_neutral","25":"negative_negative","26":"positive_positive"}
                   precision    recall  f1-score   support

negative_negative       0.44      1.00      0.61         7
 negative_neutral       0.00      0.00      0.00         4
negative_positive       0.00      0.00      0.00         1
 neutral_negative       0.00      0.00      0.00         2
  neutral_neutral       0.50      0.50      0.50   

/var/folders/xm/m4dy5bvs1fn7fb9wxjhsg4dc0000gn/T/ipykernel_10011/397373652.py:29: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  predictions = pd.read_json(output, orient="index")
/Users/valena17/azure_gpt/azure_gpt_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/valena17/azure_gpt/azure_gpt_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/valena17/azure_gpt/a